In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, Dataset
from torchvision import datasets, transforms
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import numpy as np
import sys
import copy
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_breast_cancer
import pandas as pd
import pickle

torch.manual_seed(42)

# 하이퍼파라미터 설정
batch_size = 64
epochs = 10
lr = 0.01
PCA_dim = 8
CLS_num = 2



with open('./data.pkl','rb') as file:
    data = pickle.load(file)
X = data['X']
y = data['Y']



x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)



def Fit_to_quantum(X,PCA_dim):
    pca = PCA(n_components=PCA_dim)
    X_pca = pca.fit_transform(X)
    return X_pca
    



# PyTorch Tensor로 변환
x_train_pca, y_train = torch.tensor(x_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
x_test_pca, y_test = torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)


class Feature_data_loader(Dataset):
    def __init__(self,x_train,y_train):
        self.feature1 = x_train
        temp = copy.deepcopy(x_train)
        shuffle = torch.randperm(len(temp))
        self.feature2 = temp[shuffle]
        self.y1 = y_train
        temp_y = copy.deepcopy(y_train)
        self.y2 = temp_y[shuffle]
        
    def __len__(self):
        return len(self.feature1)
    def __getitem__(self,idx):
        input1 = self.feature1[idx]
        input2 = self.feature2[idx]
        if self.y1[idx] == self.y2[idx]:
            label = torch.tensor(1.).float()
        else:
            label = torch.tensor(0.).float()
        return [input1,input2],label


# DataLoader 생성


feature_loader = DataLoader(Feature_data_loader(x_train_pca, y_train.float()),batch_size=batch_size,shuffle=True)
test_feature_loader = DataLoader(Feature_data_loader(x_test_pca, y_test.float()),batch_size=batch_size,shuffle=False)
train_loader = DataLoader(TensorDataset(x_train_pca, y_train), batch_size=batch_size, shuffle=True)
test_loader = DataLoader(TensorDataset(x_test_pca, y_test), batch_size=batch_size, shuffle=False)


In [2]:
def accuracy(pred, true):
    # 예측값이 로짓 혹은 확률값인 경우, 최대 값을 가진 인덱스를 구함 (가장 확률이 높은 클래스)
    pred = pred.detach().cpu()
    true = true.cpu()
    try:
        pred_labels = torch.argmax(pred, dim=1)
    except:
        pred_labels = torch.round(pred)
    # 예측 레이블과 실제 레이블이 일치하는 경우를 계산
    correct = (pred_labels == true).sum()
    # 정확도를 계산
    acc = correct / true.size(0)
    return acc.item()

class Early_stop_train():
    def __init__(self,model, optimizer, criterion):
        self.model = model
        self.optimizer = optimizer
        self.criterion = criterion



        self.loss_list = [1e100]
        self.stop_count = 0

    def train_model(self,train_loader,test_loader=None ,epochs=200,res = 10):
        self.model.train()
        for epoch in range(epochs):
            if self.stop_count>=res:
                break
            loss_val,_ = self.test(test_loader)
            self.loss_list.append(loss_val)

            if self.loss_list[-1]>=np.min(self.loss_list[:-1]):
                self.stop_count+=1
            else:
                self.stop_count = 0
            loss_list = []
            acc_list = []
            for X_train,y_train in train_loader:

                self.optimizer.zero_grad()
                output = self.model(X_train)

                loss = self.criterion(output.squeeze(), y_train)

                loss.backward()
                self.optimizer.step()
                loss_list.append(loss.item())
                acc = accuracy(output,y_train)
                acc_list.append(acc)

                sys.stdout.write(f"\rEpoch {epoch+1} Loss {np.mean(loss_list):4f} acc : {np.mean(acc_list):4f} stop count : {self.stop_count}")


    def test(self,test_loader):
        if test_loader is None:
            return 0,0
        else:
            #self.model.eval()
            test_loss = 0
            correct = 0
            with torch.no_grad():
                for data, target in test_loader:
                    data, target = data, target
                    output = self.model(data)
                    test_loss += self.criterion(output.squeeze(), target).item()

                    correct += accuracy(output,target)*len(output)

            print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} ({100. * correct / len(test_loader.dataset):.0f}%)')
            return test_loss,correct

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pennylane as qml
from pennylane import numpy as np
from collections import OrderedDict
import math

result_list_classical = []

# 데이터 로드 및 전처리
"""
data = load_breast_cancer()
X = data.data
y = data.target

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

pca = PCA(n_components=PCA_dim)
X_pca = pca.fit_transform(X_scaled)
X_train, X_test, y_train, y_test = train_test_split(X_pca, y, test_size=0.PCA_dim, random_state=seed)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)"""

# Pennylane 장치 설정
dev = qml.device("default.qubit", wires=PCA_dim)


def ZZFeatureMapLayer(features, wires):
    """사용자 정의 ZZFeatureMap 레이어"""
    index = 0
    for i in wires:
        qml.Hadamard(wires=i)
        qml.RZ(features[:,index], wires=i)
        index += 1

    for j in range(0, len(wires)-1):
        qml.CNOT(wires=[j, j+1])
        qml.RZ((features[:,index]), wires=j+1)
        qml.CNOT(wires=[j, j+1])
        index+=1

def ansatz(params):
    for j in range(len(params)):
        # 각 큐비트에 대해 RX, RY, RZ 회전 적용
        for i in range(len(params[0])):
            qml.RY(params[j, i, 0], wires=i)
            qml.RZ(params[j, i, 1], wires=i)

        # 인접한 큐비트 간 CNOT 게이트로 엔탱글링
        if j == len(params)-1:
            pass
        else:
            for i in range(len(params[0])-1):
                qml.CNOT(wires=[i, i+1])


# 양자 레이어 정의
@qml.qnode(dev, interface='torch', diff_method="backprop")
def QuantumLayer(features,params):
    ZZFeatureMapLayer(features, wires=range(PCA_dim))
    ansatz(params)
    return qml.probs(wires=range(math.ceil(math.log2(CLS_num))))


## 양자 커널
@qml.qnode(dev, interface='torch', diff_method="backprop")
def Kernal(features1,features2):
    ZZFeatureMapLayer(features1, wires=range(PCA_dim))
    qml.adjoint(ZZFeatureMapLayer)(features2,wires=range(PCA_dim))
    return qml.probs(wires=range(PCA_dim))


class Feature_model(nn.Module):
    def __init__(self):
        super(Feature_model,self).__init__()
        self.cls = nn.Sequential(OrderedDict([('cls1', nn.Linear(PCA_dim,PCA_dim*8)),
                                              ('relu1', nn.ReLU()),('cls2', nn.Linear(PCA_dim*8,PCA_dim*8)),
                                              ('relu2', nn.ReLU()),('cls3', nn.Linear(PCA_dim*8,PCA_dim*8)),
                                              ('relu3', nn.ReLU()),('cls4', nn.Linear(PCA_dim*8,PCA_dim*8)),
                                              ('relu4', nn.ReLU()),('cls5', nn.Linear(PCA_dim*8,PCA_dim*2-1)),
                                              ('sigmoid', nn.ReLU())]))
        self.Kernal = Kernal
    def forward(self,inputs):
        epsilon = 1e-6
        input1 = inputs[0]
        input2 = inputs[1]
        input1 = self.cls(input1)*np.pi
        input2 = self.cls(input2)*np.pi
        output = self.Kernal(input1,input2)
        output = output.type(torch.float32)
        return output[:,0].clamp(min=epsilon, max=1-epsilon)





# 하이브리드 모델 정의
class HybridModel(nn.Module):
    def __init__(self):
        super(HybridModel, self).__init__()
        self.cls = feature_model.cls

        self.quantum_layer = QuantumLayer
        self.Q_params = nn.Parameter((torch.rand([PCA_dim,PCA_dim,2])*2-1)*np.pi,requires_grad=True)
    def forward(self, x):
        x = self.cls(x)*np.pi
        #print(qml.draw(self.quantum_layer)(x,self.Q_params))
        quantum_output = self.quantum_layer(x,self.Q_params)
        quantum_output = quantum_output.type(torch.float32)
        return torch.log(quantum_output)

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.cls_layer_1 = nn.Linear(PCA_dim,PCA_dim*PCA_dim)
        self.cls_layer_2 = nn.Linear(PCA_dim*PCA_dim,PCA_dim*PCA_dim-1)
        self.output_layer = nn.Linear(PCA_dim*PCA_dim-1,PCA_dim)
    def forward(self, x):
        x = self.cls_layer_1(x)
        x = nn.ReLU()(x)
        x = self.cls_layer_2(x)
        x = nn.ReLU()(x)
        output = self.output_layer(x)
        return output
# 모델, 손실 함수, 최적화 설정


feature_model = Feature_model(); criterion = nn.BCELoss()
#model = Model(); criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(feature_model.parameters(), lr=0.001)


# 모델 학습 및 평가
train_process = Early_stop_train(feature_model, optimizer, criterion)
train_process.train_model(feature_loader,test_feature_loader,epochs=50,res=15)
"""
model = HybridModel(); criterion = nn.NLLLoss()
for param in model.cls.parameters():
    param.requires_grad = False
#model.load_state_dict(para_dict)
optimizer = optim.Adam(model.parameters(), lr=0.01)
print('\n\n Test start \n\n')
train_process = Early_stop_train(model, optimizer, criterion)
train_process.train_model(train_loader,test_loader,epochs=50,res=5)

_,acc = train_process.test(test_loader)
result_list_classical.append(acc)
print(f"Test Accuracy: {acc:.2f}")"""




Test set: Average loss: 12.3056, Accuracy: 153.99999904632568/300 (51%)
Epoch 1 Loss 1.710618 acc : 0.491477 stop count : 0
Test set: Average loss: 4.6124, Accuracy: 157.99999952316284/300 (53%)
Epoch 2 Loss 0.849652 acc : 0.566098 stop count : 0
Test set: Average loss: 4.0512, Accuracy: 187.99999952316284/300 (63%)
Epoch 3 Loss 0.747656 acc : 0.602841 stop count : 0
Test set: Average loss: 3.2585, Accuracy: 198.0000011920929/300 (66%)
Epoch 4 Loss 0.632870 acc : 0.662879 stop count : 0
Test set: Average loss: 2.8490, Accuracy: 223.00000071525574/300 (74%)
Epoch 5 Loss 0.562981 acc : 0.708712 stop count : 0
Test set: Average loss: 2.4969, Accuracy: 230.00000071525574/300 (77%)
Epoch 6 Loss 0.507302 acc : 0.754545 stop count : 0
Test set: Average loss: 2.2532, Accuracy: 242.00000071525574/300 (81%)
Epoch 7 Loss 0.473032 acc : 0.780682 stop count : 0
Test set: Average loss: 2.1640, Accuracy: 249.99999976158142/300 (83%)
Epoch 8 Loss 0.446762 acc : 0.797254 stop count : 0
Test set: Avera

KeyboardInterrupt: 

In [5]:
from tqdm import tqdm
x_train = torch.tensor(x_train).float()
num_data = x_train.shape[0]
kernel_matrix = torch.zeros((num_data, num_data), dtype=torch.float32)

for i in tqdm(range(num_data)):
    data = torch.stack([x_train[i]]*num_data)
    output = feature_model([data,x_train])
    kernel_matrix[i] = output

 12%|█▏        | 86/700 [01:23<09:55,  1.03it/s]


KeyboardInterrupt: 

In [ ]:
labels = torch.tensor(y_train).float()
labels = 2*labels-1
alpha = torch.tensor([0.5]*num_data,requires_grad=True)
optimizer = torch.optim.Adam([alpha], lr=0.001)
def objective_function(alpha, kernel_matrix, labels):
    """SVM의 쌍대 목적 함수"""
    L = 0.5 * torch.dot(alpha, torch.mv(kernel_matrix, alpha)) - torch.sum(alpha)
    # 제약 조건을 유지하기 위해 레이블과 alpha의 곱의 합은 0이어야 합니다.
    constraint = torch.dot(alpha, labels)
    loss = -L + 1e4 * constraint ** 2
    print(loss,1e4 * constraint ** 2)
    return loss  # 제약조건에 큰 페널티를 적용

# 훈련 과정
epochs = 5000
for epoch in range(epochs):
    optimizer.zero_grad()
    loss = objective_function(alpha, kernel_matrix, labels)
    loss.backward()
    optimizer.step()
    alpha.data.clamp_(0)  # alpha는 0 이상이어야 함

print("Optimized alphas:", alpha.data)

C:\Users\pop75\AppData\Local\Temp\ipykernel_30240\1268779803.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(y_train).float()


tensor(11626.8711, grad_fn=<AddBackward0>) tensor(40000., grad_fn=<MulBackward0>)
tensor(-11467.4785, grad_fn=<AddBackward0>) tensor(16900.4082, grad_fn=<MulBackward0>)
tensor(-24483.9355, grad_fn=<AddBackward0>) tensor(3879.0730, grad_fn=<MulBackward0>)
tensor(-28358.6680, grad_fn=<AddBackward0>) tensor(0.1115, grad_fn=<MulBackward0>)
tensor(-25809.5742, grad_fn=<AddBackward0>) tensor(2546.0581, grad_fn=<MulBackward0>)
tensor(-21156.7012, grad_fn=<AddBackward0>) tensor(7197.2168, grad_fn=<MulBackward0>)
tensor(-18112.3242, grad_fn=<AddBackward0>) tensor(10241.2979, grad_fn=<MulBackward0>)
tensor(-17972.2812, grad_fn=<AddBackward0>) tensor(10382.1631, grad_fn=<MulBackward0>)
tensor(-20125.0586, grad_fn=<AddBackward0>) tensor(8231.0068, grad_fn=<MulBackward0>)
tensor(-23266.3809, grad_fn=<AddBackward0>) tensor(5091.8472, grad_fn=<MulBackward0>)
tensor(-26152.6465, grad_fn=<AddBackward0>) tensor(2208.0950, grad_fn=<MulBackward0>)
tensor(-27942.9102, grad_fn=<AddBackward0>) tensor(420.529

KeyboardInterrupt: 

In [ ]:
import pennylane as qml
import torch
import numpy as np



# 테스트 데이터와 훈련 데이터 간의 양자 커널 행렬 계산
x_test = torch.tensor(x_test).float()
num_test = x_test.size(0)
test_kernel_matrix = torch.zeros((num_test, num_data), dtype=torch.float32)

for i in tqdm(range(num_data)):
    data = torch.stack([x_train[i]]*num_test)
    output = feature_model([data,x_test])
    test_kernel_matrix[:,i] = output

# 훈련된 모델을 사용하여 테스트 데이터의 클래스 예측
predictions = torch.sign(torch.mv(test_kernel_matrix, alpha * labels))

print("Predictions:", predictions)

C:\Users\pop75\AppData\Local\Temp\ipykernel_30240\3424979452.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x_test = torch.tensor(x_test).float()
 75%|███████▌  | 528/700 [02:38<01:06,  2.59it/s]